In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14506,2022-01-14T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1307,165,1472,158121,...,DECEDUTI: N. 6 IL 13/01/22 - N. 11 IL 12/01/22...,15.0,NaN,NaN,440446.0,64363.0,4046712.0,5031625.0,ITG,ITG1
14507,2022-01-14T17:00:00,ITA,9,Toscana,43.769231,11.255889,1189,126,1315,184563,...,NaN,9.0,NaN,NaN,432337.0,127030.0,5737778.0,4245787.0,ITI,ITI1
14508,2022-01-14T17:00:00,ITA,10,Umbria,43.106758,12.388247,204,12,216,34703,...,- Si fa presente che 5 dei ricoveri non UTI ap...,0.0,NaN,NaN,96205.0,27847.0,1406837.0,1751835.0,ITI,ITI2
14509,2022-01-14T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,63,6,69,5925,...,NaN,0.0,NaN,NaN,14577.0,8471.0,130034.0,265373.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14506,2022-01-14T17:00:00,19,Sicilia,1307,165,1472,158121,159593,9127,10023,...,504809,9078337,3537557.0,15.0,440446.0,64363.0,4046712.0,5031625.0,ITG,ITG1
14507,2022-01-14T17:00:00,9,Toscana,1189,126,1315,184563,185878,4964,11859,...,559367,9983565,3907096.0,9.0,432337.0,127030.0,5737778.0,4245787.0,ITI,ITI1
14508,2022-01-14T17:00:00,10,Umbria,204,12,216,34703,34919,-125,2125,...,124052,3158672,589885.0,0.0,96205.0,27847.0,1406837.0,1751835.0,ITI,ITI2
14509,2022-01-14T17:00:00,2,Valle d'Aosta,63,6,69,5925,5994,146,587,...,23048,395407,119027.0,0.0,14577.0,8471.0,130034.0,265373.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2022-01-14', '2022-01-13')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-14T17:00:00,1,Piemonte,1930,147,2077,160774,162851,2257,14609,...,670701,12401294,3317386.0,18.0,422585.0,248116.0,4321177.0,8080117.0,ITC,ITC1
1,2022-01-14T17:00:00,2,Valle d'Aosta,63,6,69,5925,5994,146,587,...,23048,395407,119027.0,0.0,14577.0,8471.0,130034.0,265373.0,ITC,ITC2
2,2022-01-14T17:00:00,3,Lombardia,3469,262,3731,551973,555704,-22553,33856,...,1714922,27463252,7226692.0,14.0,1139787.0,575135.0,13751382.0,13711870.0,ITC,ITC4
3,2022-01-14T17:00:00,5,Veneto,1516,188,1704,230778,232482,8337,18357,...,848874,21190282,2545781.0,18.0,643952.0,204922.0,8640854.0,12549428.0,ITH,ITH3
4,2022-01-14T17:00:00,6,Friuli Venezia Giulia,369,44,413,44637,45050,2216,4584,...,200559,4841306,1003016.0,7.0,153549.0,47010.0,2764110.0,2077196.0,ITH,ITH4
5,2022-01-14T17:00:00,7,Liguria,691,43,734,23963,24697,1961,5438,...,211077,3851627,1102844.0,4.0,177969.0,33108.0,2096064.0,1755563.0,ITC,ITC3
6,2022-01-14T17:00:00,8,Emilia-Romagna,2312,148,2460,274333,276793,17329,20346,...,771295,12445735,2481206.0,7.0,640119.0,131176.0,7678874.0,4766861.0,ITH,ITH5
7,2022-01-14T17:00:00,9,Toscana,1189,126,1315,184563,185878,4964,11859,...,559367,9983565,3907096.0,9.0,432337.0,127030.0,5737778.0,4245787.0,ITI,ITI1
8,2022-01-14T17:00:00,10,Umbria,204,12,216,34703,34919,-125,2125,...,124052,3158672,589885.0,0.0,96205.0,27847.0,1406837.0,1751835.0,ITI,ITI2
9,2022-01-14T17:00:00,11,Marche,271,58,329,12422,12751,1850,4257,...,171633,2204552,1325723.0,0.0,168292.0,3341.0,1737138.0,467414.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-13T17:00:00,1,Piemonte,1930,146,2076,158518,160594,3780,14741,...,656092,12297287,3297724.0,16.0,420368.0,235724.0,4306367.0,7990920.0,ITC,ITC1
1,2022-01-13T17:00:00,2,Valle d'Aosta,53,7,60,5788,5848,307,574,...,22461,392621,118621.0,2.0,14554.0,7907.0,129838.0,262783.0,ITC,ITC2
2,2022-01-13T17:00:00,3,Lombardia,3452,257,3709,574548,578257,27761,39683,...,1681066,27249089,7196405.0,21.0,1127963.0,553103.0,13690844.0,13558245.0,ITC,ITC4
3,2022-01-13T17:00:00,5,Veneto,1410,191,1601,222544,224145,8950,17956,...,830517,21046121,2539358.0,18.0,636441.0,194076.0,8612108.0,12434013.0,ITH,ITH3
4,2022-01-13T17:00:00,6,Friuli Venezia Giulia,369,41,410,42424,42834,2079,4039,...,195986,4810175,999674.0,7.0,151781.0,44205.0,2752505.0,2057670.0,ITH,ITH4
5,2022-01-13T17:00:00,7,Liguria,676,40,716,22020,22736,2252,5692,...,205639,3820122,1097157.0,4.0,176396.0,29243.0,2089748.0,1730374.0,ITC,ITC3
6,2022-01-13T17:00:00,8,Emilia-Romagna,2258,151,2409,257055,259464,16779,20648,...,750971,12376070,2474069.0,14.0,631185.0,119786.0,7646878.0,4729192.0,ITH,ITH5
7,2022-01-13T17:00:00,9,Toscana,1173,123,1296,179618,180914,2200,13151,...,547508,9912470,3889487.0,7.0,427308.0,120200.0,5715108.0,4197362.0,ITI,ITI1
8,2022-01-13T17:00:00,10,Umbria,214,12,226,34818,35044,720,2068,...,121927,3142570,587132.0,0.0,95291.0,26636.0,1403083.0,1739487.0,ITI,ITI2
9,2022-01-13T17:00:00,11,Marche,266,59,325,10576,10901,-52,1955,...,167376,2185228,1310463.0,1.0,167376.0,0.0,1728863.0,456365.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2022-01-14T17:00:00,1,Piemonte,1930,147,2077,160774,162851,2257,14609,...,12401294,3317386.0,18.0,422585.0,248116.0,4321177.0,8080117.0,ITC,ITC1,4.341375e+06
1,2022-01-14T17:00:00,2,Valle d'Aosta,63,6,69,5925,5994,146,587,...,395407,119027.0,0.0,14577.0,8471.0,130034.0,265373.0,ITC,ITC2,1.255010e+05
2,2022-01-14T17:00:00,3,Lombardia,3469,262,3731,551973,555704,-22553,33856,...,27463252,7226692.0,14.0,1139787.0,575135.0,13751382.0,13711870.0,ITC,ITC4,1.010397e+07
3,2022-01-14T17:00:00,5,Veneto,1516,188,1704,230778,232482,8337,18357,...,21190282,2545781.0,18.0,643952.0,204922.0,8640854.0,12549428.0,ITH,ITH3,4.907704e+06
4,2022-01-14T17:00:00,6,Friuli Venezia Giulia,369,44,413,44637,45050,2216,4584,...,4841306,1003016.0,7.0,153549.0,47010.0,2764110.0,2077196.0,ITH,ITH4,1.211357e+06
5,2022-01-14T17:00:00,7,Liguria,691,43,734,23963,24697,1961,5438,...,3851627,1102844.0,4.0,177969.0,33108.0,2096064.0,1755563.0,ITC,ITC3,1.543127e+06
6,2022-01-14T17:00:00,8,Emilia-Romagna,2312,148,2460,274333,276793,17329,20346,...,12445735,2481206.0,7.0,640119.0,131176.0,7678874.0,4766861.0,ITH,ITH5,4.467118e+06
7,2022-01-14T17:00:00,9,Toscana,1189,126,1315,184563,185878,4964,11859,...,9983565,3907096.0,9.0,432337.0,127030.0,5737778.0,4245787.0,ITI,ITI1,3.722729e+06
8,2022-01-14T17:00:00,10,Umbria,204,12,216,34703,34919,-125,2125,...,3158672,589885.0,0.0,96205.0,27847.0,1406837.0,1751835.0,ITI,ITI2,8.802850e+05
9,2022-01-14T17:00:00,11,Marche,271,58,329,12422,12751,1850,4257,...,2204552,1325723.0,0.0,168292.0,3341.0,1737138.0,467414.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2022-01-14T17:00:00,1,Piemonte,1930,147,2077,160774,162851,2257,14609,...,8080117.0,ITC,ITC1,4.341375e+06,Piemonte,1930,6794,147,628,99
1,2022-01-14T17:00:00,2,Valle d'Aosta,63,6,69,5925,5994,146,587,...,265373.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,63,99,6,33,0
2,2022-01-14T17:00:00,3,Lombardia,3469,262,3731,551973,555704,-22553,33856,...,13711870.0,ITC,ITC4,1.010397e+07,Lombardia,3469,10457,262,1530,0
3,2022-01-14T17:00:00,5,Veneto,1516,188,1704,230778,232482,8337,18357,...,12549428.0,ITH,ITH3,4.907704e+06,Veneto,1516,6000,188,1000,0
4,2022-01-14T17:00:00,6,Friuli Venezia Giulia,369,44,413,44637,45050,2216,4584,...,2077196.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,369,1277,44,175,0
5,2022-01-14T17:00:00,7,Liguria,691,43,734,23963,24697,1961,5438,...,1755563.0,ITC,ITC3,1.543127e+06,Liguria,691,1827,43,220,12
6,2022-01-14T17:00:00,8,Emilia-Romagna,2312,148,2460,274333,276793,17329,20346,...,4766861.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,2312,9001,148,889,0
7,2022-01-14T17:00:00,9,Toscana,1189,126,1315,184563,185878,4964,11859,...,4245787.0,ITI,ITI1,3.722729e+06,Toscana,1189,5033,126,570,12
8,2022-01-14T17:00:00,10,Umbria,204,12,216,34703,34919,-125,2125,...,1751835.0,ITI,ITI2,8.802850e+05,Umbria,204,662,12,86,41
9,2022-01-14T17:00:00,11,Marche,271,58,329,12422,12751,1850,4257,...,467414.0,ITI,ITI3,1.518400e+06,Marche,271,1012,58,256,49


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2022-01-14T17:00:00,3,Lombardia,3469,262,3731,551973,555704,-22553,33856,...,17.12,-22575,-5827,214163,30287.0,115,56294,15.81,0.33508,16.97
1,2022-01-14T17:00:00,8,Emilia-Romagna,2312,148,2460,274333,276793,17329,20346,...,16.65,17278,-302,69665,7137.0,24,2971,29.21,0.45546,17.27
2,2022-01-14T17:00:00,15,Campania,1191,79,1270,227139,228409,9455,20206,...,11.30,9418,-4245,100228,18847.0,26,10725,20.16,0.34923,13.90
3,2022-01-14T17:00:00,5,Veneto,1516,188,1704,230778,232482,8337,18357,...,18.80,8234,401,144161,6423.0,29,9991,12.73,0.37404,17.30
4,2022-01-14T17:00:00,12,Lazio,1675,204,1879,202602,204481,10078,15307,...,21.63,10023,5035,102132,7767.0,25,5204,14.99,0.26096,11.43
5,2022-01-14T17:00:00,1,Piemonte,1930,147,2077,160774,162851,2257,14609,...,23.41,2256,-132,104007,19662.0,21,12331,14.05,0.33651,15.45
6,2022-01-14T17:00:00,9,Toscana,1189,126,1315,184563,185878,4964,11859,...,22.11,4945,-1292,71095,17609.0,24,6871,16.68,0.31856,15.03
7,2022-01-14T17:00:00,19,Sicilia,1307,165,1472,158121,159593,9127,10023,...,19.57,9118,-1331,50567,49829.0,21,875,19.82,0.20173,10.16
8,2022-01-14T17:00:00,16,Puglia,530,56,586,84073,84659,21758,9757,...,11.36,21720,6539,75448,10818.0,8,2640,12.93,0.24342,9.86
9,2022-01-14T17:00:00,7,Liguria,691,43,734,23963,24697,1961,5438,...,19.55,1943,-254,31505,5687.0,21,3456,17.26,0.35240,13.68


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,23,67,90,25.56
1,Friuli Venezia Giulia,44,131,175,25.14
2,Piemonte,147,481,628,23.41
3,Marche,58,198,256,22.66
4,Abruzzo,41,140,181,22.65
5,Toscana,126,444,570,22.11
6,Lazio,204,739,943,21.63
7,Sicilia,165,678,843,19.57
8,Liguria,43,177,220,19.55
9,Veneto,188,812,1000,18.80


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))